# Обучение NLP-пайплайна (TF-IDF и K-Means)

**Цель:** Создать и сохранить готовый к использованию NLP-пайплайн для анализа и кластеризации логов.

## Введение: Как найти смысл в хаосе логов?

Логи — это неструктурированный текст. Чтобы машина могла их анализировать, нам нужно превратить их в числа, сохранив при этом их семантический смысл. После этого мы можем применить алгоритмы кластеризации, чтобы автоматически найти группы похожих ошибок.

Наш пайплайн состоит из двух этапов:

1.  **Векторизация (TF-IDF):** Создание "карты мира", где каждое слово имеет свои координаты.
2.  **Кластеризация (K-Means):** Нахождение "столиц" (центров групп ошибок) на этой карте.

Мы обучим обе модели и сохраним их как `tfidf_vectorizer.pkl` и `kmeans_model.pkl`.

## Шаг 1: Подготовка и генерация данных

В реальной системе мы бы взяли сотни тысяч логов из продакшена за последний месяц, чтобы создать репрезентативный корпус текстов. Для демонстрации мы сгенерируем небольшой, но разнообразный набор типовых ошибок.

In [1]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import os

MODELS_DIR = "trained_models"
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

print("Генерация большого набора логов для обучения NLP-пайплайна...")
logs_corpus = [
    "Database connection refused by host 127.0.0.1",
    "Failed to authenticate user 'guest'",
    "java.lang.NullPointerException: Cannot invoke 'user.getName()' because 'user' is null",
    "OutOfMemoryError: Java heap space",
    "[ERROR] DB Connection timeout after 3000ms",
    "Invalid API key provided for user 12345",
    "User authentication failed for admin from IP 192.168.1.1",
    "Could not connect to Redis server on port 6379"
] * 100

print(f"Создан обучающий корпус из {len(logs_corpus)} лог-сообщений.")

Генерация большого набора логов для обучения NLP-пайплайна...
Создан обучающий корпус из 800 лог-сообщений.


## Шаг 2: Обучение TF-IDF Векторизатора (`tfidf_vectorizer.pkl`)

**TF-IDF (Term Frequency-Inverse Document Frequency)** придает больший вес словам, которые редки во всем наборе данных, но часто встречаются в конкретном сообщении, что идеально для выделения ключевых слов в ошибках.

Результатом будет обученный "словарь" и математическая модель для преобразования любого нового лога в вектор.

In [2]:
print("Обучение векторизатора 'tfidf_vectorizer.pkl'...")
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)

# Обучаем векторизатор на нашем корпусе и сразу трансформируем данные для следующего шага
X_train = tfidf_vectorizer.fit_transform(logs_corpus)

vectorizer_path = os.path.join(MODELS_DIR, "tfidf_vectorizer.pkl")
with open(vectorizer_path, 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print(f" -> Векторизатор TF-IDF обучен на {len(tfidf_vectorizer.get_feature_names_out())} уникальных словах и сохранен в '{vectorizer_path}'")

Обучение векторизатора 'tfidf_vectorizer.pkl'...
 -> Векторизатор TF-IDF обучен на 37 уникальных словах и сохранен в 'trained_models\tfidf_vectorizer.pkl'


## Шаг 3: Обучение K-Means Кластеризатора (`kmeans_model.pkl`)

**K-Means** — это алгоритм, который находит центры групп (кластеров) на "карте", созданной TF-IDF. Обученная модель сможет для любого нового лога (представленного в виде вектора) мгновенно сказать, к какой из заранее определенных групп ошибок он относится.

In [3]:
print("Обучение модели 'kmeans_model.pkl'...")
# В реальной системе количество кластеров (n_clusters) подбирается экспериментально
N_CLUSTERS = 3
kmeans_model = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init='auto')

# Обучаем модель на векторизованном корпусе
kmeans_model.fit(X_train)

kmeans_path = os.path.join(MODELS_DIR, "kmeans_model.pkl")
with open(kmeans_path, 'wb') as f:
    pickle.dump(kmeans_model, f)

print(f" -> Модель K-Means обучена на {N_CLUSTERS} кластера и сохранена в '{kmeans_path}'")

Обучение модели 'kmeans_model.pkl'...
 -> Модель K-Means обучена на 3 кластера и сохранена в 'trained_models\kmeans_model.pkl'


## Заключение

**Процесс обучения завершен.**

В директории `trained_models/` теперь находятся все необходимые, предварительно обученные артефакты. Основное приложение может теперь загружать эти файлы и выполнять анализ новых данных быстро и эффективно.